# Workflow 1, Module 3a

Given a disease, first find chemical agents that make the disease worse. Then look for genes that decrease the amount of or response to the toxicant, and try to increase the amount or action of those genes.

In [14]:
input_disease = 'MONDO:0005148'  #Type 2 diabetes

## Functions

In [15]:
#To make nicer looking outputs
from IPython.core.display import display, HTML
import requests
import pandas as pd

#Load some functions for parsing quick output
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from gg_functions import parse_answer, get_view_url, quick

In [29]:
#Note that we're demanding that the chemical_substance that comes back is already listed as a drug.

def make_question(curie,is_drug=True):
    mq=  {
        "machine_question":
        {
        "nodes": [
            {
                "id": "n0",
                "type": "disease",
                "curie": curie
            },
            {
                "id": "n1",
                "type": "chemical_substance"
            },
            {
                "id": "n2",
                "type": "gene"
            },
            {
                "id": "n3",
                "type": "chemical_substance"
            }
        ],
        "edges": [
            {
                "id": "e1",
                "source_id": "n1",
                "target_id": "n0",
                "type": "contributes_to"
            },
            {
                "id": "e2",
                "source_id": "n2",
                "target_id": "n1",
                "type": [
                    "increases_degradation_of",
                    "decreases_abundance_of",
                    "decreases_response_to",
                    "increases_uptake_of"
                ]
            },
            {
                "id": "e3",
                "source_id": "n3",
                "target_id": "n2",
                "type": [
                    "increases_activity_of",
                    "increases_expression_of",
                    "decreases_degradation_of",
                    "increases_stability_of",
                    "increases_synthesis_of",
                    "increases_secretion_of"
                ]
            }
        ]
        }
    }
    if is_drug:
        mq['machine_question']['nodes'][3]['drug']=True
    return mq


In [30]:
single_q = make_question(input_disease)
single_answer = quick(single_q)

Return Status: 200


In [31]:
df = parse_answer(single_answer,node_list=['n1','n2','n3','n4'],edge_list=['e1','e2','e3'],node_properties=['name'],edge_properties=['type'])

In [32]:
from IPython.display import display
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

,e1 - type,e2 - type,e3 - type,n1 - name,n2 - name,n3 - name,score
0,contributes_to,increases_uptake_of,increases_expression_of,glucose,SLC2A4,metformin,0.953679
1,contributes_to,increases_uptake_of,increases_secretion_of,glucose,INS,metformin,0.874051
2,contributes_to,decreases_response_to,increases_activity_of,streptozocin,CAT,metformin,0.709781
3,contributes_to,decreases_response_to,increases_expression_of,glucose,SOD2,ethanol,0.699207
4,contributes_to,decreases_response_to,increases_expression_of,glucose,UCP1,rosiglitazone,0.683096
5,contributes_to,increases_uptake_of,increases_expression_of,glucose,SLC2A4,rosiglitazone,0.682681
6,contributes_to,decreases_response_to,increases_activity_of,glutathione,CAT,metformin,0.675596
7,contributes_to,decreases_response_to,increases_expression_of,glucose,RELA,ethanol,0.665503
8,contributes_to,increases_uptake_of,increases_expression_of,glucose,INS,cortisol,0.663066
9,contributes_to,decreases_response_to,increases_expression_of,glucose,HMOX1,ethanol,0.662286


In [33]:
view_url = get_view_url(single_answer)
display(HTML(f'<a href={view_url}>View Answer in ROBOKOP</a>'))

Just to make this a little easier to read, let's just keep the first row whenever we find a new chemical.  So for instance, we'll keep the first metformin above, but not any others. This potentially hides some extra evidence for a chemical, so use with caution, but it makes it easier to get an overall view of what chemicals are found.  It's ok to get the first row because the frame is already sorted by score.

In [34]:
singledf = df.groupby('n3 - name ').nth(0).reset_index().sort_values(by='score',ascending=False)
from IPython.display import display
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(singledf)

,n3 - name,e1 - type,e2 - type,e3 - type,n1 - name,n2 - name,score
43,metformin,contributes_to,increases_uptake_of,increases_expression_of,glucose,SLC2A4,0.953679
32,ethanol,contributes_to,decreases_response_to,increases_expression_of,glucose,SOD2,0.699207
64,rosiglitazone,contributes_to,decreases_response_to,increases_expression_of,glucose,UCP1,0.683096
23,cortisol,contributes_to,increases_uptake_of,increases_expression_of,glucose,INS,0.663066
68,streptozocin,contributes_to,decreases_response_to,increases_expression_of,glucose,HMOX1,0.657291
13,bisphenol A,contributes_to,increases_uptake_of,increases_expression_of,glucose,INS1,0.620630
52,olanzapine,contributes_to,increases_uptake_of,increases_expression_of,glucose,INS,0.619725
2,ATP,contributes_to,increases_uptake_of,increases_secretion_of,glucose,INS1,0.618002
36,hydrogen peroxide,contributes_to,decreases_response_to,increases_expression_of,glucose,SOD2,0.610051
34,glyburide,contributes_to,increases_uptake_of,increases_secretion_of,glucose,INS,0.603946


### Notes on the rows that are not already known treatments 

Ethanol: Ethanol can reduce glucose levels, but it's a complicated picture, because it can also increase glucose.  Long-term ethanol can also lead to reduced insulin sensititivy.

cortisol: Stress hormone.  Tends to be contributor to T2D.  It's not surprising to find contributors from this query.  Typically a compound that is cleaned up by some gene or process will also stimulate that gene or process by its presence. This query is susceptible to this kind of feedback loop.

streptozocin: can induce diabetes

bisphenol A: exposure may be linked to an increase in diabetes

olanzapine: can disregulate glucose, leading to diabetes

ATP: ATP is involved in cell-energy cycles along with glucose

hydrogen peroxide: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3216925/  "Our results demonstrate that depending on its concentration, H2O2 can have the positive or negative effect on insulin signal transduction in H4IIEC hepatocytes, suggesting that the concentration of intracellular ROS may be a major factor in determining whether ROS impair or enhance insulin signaling."

Melatonin: Hormone involved in sleep (and used to treat insomnia).  There's some association with T2D, but it's complicated: https://link.springer.com/article/10.1007/s00125-017-4255-x

rotenone: an isoflavone used as a pesticide, but: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5783883/ "Rotenone ameliorated hyperglycaemia and insulin resistance in db/db mice. It induced glucose consumption and glycolysis and reduced hepatic glucose output. Rotenone also activated AMPK. Furthermore, it remained effective with AMPK inactivation."

simvastatin: statins tend to increase diabetic risk, so this probably goes in the contributors column. 

terbutaline: beta2 agonist for asthma.  Known to cause hyperglycemia and exacerbate diabetes.  

curcumin: a suppliment from turmeric.   There's lots of papers about its supposedly helpful role in diabetes, but I can't tell what to make of them.

nitric oxide: NO signalling is known to be reduced in diabetes. Not clear what that means re: therapy

quercetin: a flavonoid. https://www.sciencedirect.com/science/article/pii/S0753332218349187 "In vitro and in vivo evidence that quercetin protects against diabetes and its complications: A systematic review of the literature"

all-trans-retinoic acid: treatment for acne, and some forms of leukemia.  https://www.ncbi.nlm.nih.gov/pubmed/27050967 "All-trans retinoic acid prevents the development of type 1 diabetes by affecting the levels of interferon gamma and interleukin 4 in streptozotocin-induced murine diabetes model."

sodium arsenite: exposure leads to diabetes?

clofibrate: discontinued cholesterol treatment

lithium chloride: https://www.ncbi.nlm.nih.gov/pubmed/27241942 "Protective effect of lithium chloride against hypoglycemia-induced apoptosis in neuronal PC12 cell." https://www.ncbi.nlm.nih.gov/pubmed/8460950 "Lithium restores glycogen synthesis from glucose in hepatocytes from diabetic rats."

benzo[a]pyrene: maybe an environmental contributor?

progesterone: female sex hormone.  A lot of work in this area, but unclear what the relationship is.

valproic acid: anticonvulsant. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5483554/ "Does Valproic Acid Have Potential in the Treatment of Diabetes Mellitus?"  https://www.ncbi.nlm.nih.gov/pubmed/27079868 "Valproic Acid Improves Glucose Homeostasis by Increasing Beta-Cell Proliferation, Function, and Reducing its Apoptosis through HDAC Inhibition in Juvenile Diabetic Rat."

ritodrine: used to stop premature labor, can worsen diabetes or lead to diabetic ketoacidosis

lansoprozole: protein pump inhibitor for GERD. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4549663/ "Although PPIs may be possible candidates for a new approach in the therapy of diabetes, a prospective, long-term, randomized, double-blind, placebo-controlled study is needed to establish the effect of PPIs on glycemic control in a large number of patients with T2DM."  

....
And many more
....



The run above assumed that we wanted drugs out.  Maybe we wanted to have a broader net into any kind of chemicals.  We can turn off the drug property.

In [35]:
single_q_all_chem = make_question(input_disease,is_drug=False)
acanswer = quick(single_q_all_chem)
acdf = parse_answer(acanswer,node_list=['n1','n2','n3','n4'],edge_list=['e1','e2','e3'],node_properties=['name'],edge_properties=['type'])

Return Status: 200


In [36]:
view_acurl = get_view_url(acanswer)
display(HTML(f'<a href={view_acurl}>View Answer in ROBOKOP</a>'))

In [39]:
acsingledf = acdf.groupby('n3 - name ').nth(0).reset_index().sort_values(by='score',ascending=False)
from IPython.display import display
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(acsingledf)

,n3 - name,e1 - type,e2 - type,e3 - type,n1 - name,n2 - name,score
25,glucose,contributes_to,increases_uptake_of,increases_activity_of,glucose,INS,1.511734
16,cholesterol,contributes_to,increases_uptake_of,increases_activity_of,glucose,MYLK,1.127006
39,metformin,contributes_to,increases_uptake_of,increases_expression_of,glucose,SLC2A4,0.953679
18,copper atom,contributes_to,increases_uptake_of,increases_secretion_of,zinc atom,IL1B,0.824211
67,zinc atom,contributes_to,increases_uptake_of,increases_expression_of,iron atom,TNF,0.797488
14,cadmium atom,contributes_to,decreases_response_to,increases_expression_of,zinc atom,PTGS2,0.782764
30,iron atom,contributes_to,increases_uptake_of,increases_secretion_of,zinc atom,IL6,0.751148
34,manganese atom,contributes_to,increases_uptake_of,increases_expression_of,zinc atom,IL6,0.716666
24,ethanol,contributes_to,decreases_response_to,increases_expression_of,glucose,SOD2,0.699207
51,rosiglitazone,contributes_to,decreases_response_to,increases_expression_of,glucose,UCP1,0.683096
